## Setup

In [ ]:
class MyError(Exception):
    pass

import random

from tqdm import tqdm

from datetime import datetime
import time
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

import pandas as pd
import numpy as np
import re
from datetime import datetime
from io import StringIO
import time

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

## Step 1: Preprocess log data and establish reference time stamps for think-aloud transcripts

In [ ]:
# Functions
def process_timestamp_zoom(s):
    x = time.strptime(s, '%H:%M:%S.%f')
    ans = 3600* x.tm_hour + 60* x.tm_min + x.tm_sec + float('.'+s.split('.')[-1])
    return ans

In [ ]:
# User ID Crosswalk
crosswalk = {
    'Stu_9a771c37547c1ce5bb0e3ccd2ffa890a': 'user_1',
    'Stu_ef57d8fdab9d03a879b85fabdb5ce8c8': 'user_2',
    'Stu_12784370c142151213cedf0d527455f9': 'user_2',
    'Stu_b0e687db63e81cfbdd64f22804c5967d': 'user_3',
    'Stu_651e714c97d469adf89a47bb73e81fdb': 'user_4',
    'Stu_954e7ff89b99dedcd9aa613308a3b2ab': 'user_5',
    'Stu_1279946571c2fb21a88d1f22340d6a21': 'user_6',
    'Stu_a02379c766c89e55794be249dee8101a': 'user_7',
    'Stu_eeea2cac9ae40df584566c798a0384e7': 'user_8',
    'Stu_187d5dc77c2259af31b59badf210161b': 'user_9',
    'Stu_6ae9d35793ea37302b302dee4b4d0c19': 'user_10'
}

df = pd.read_csv('logs/ds5371_tx_All_Data_7671_2023_0520_042939.txt', sep='\t')

df.sort_values(by = ['Anon Student Id', 'Time'])

df['Time'] = df.Time.map(lambda s: datetime.strptime(s, '%Y-%m-%d %H:%M:%S'))
df = df.sort_values(by = 'Time').copy()

df = df[df['Anon Student Id'].isin(crosswalk.keys())].copy()

df['Participant'] = df['Anon Student Id'].map(crosswalk)

df = df.sort_values(by = ['Participant', 'Time'])

df2 = df[['Transaction Id', 'Participant', 'Time', 'Step Name', 'Selection', 'Action', 'Input', 'Outcome']]

df2.to_csv('transaction-ref.csv', index=False) # For reference

# Hand-coded time references for synchronizing 
# think-aloud and tutor timestamps, with some users having 
# multiple recordings.

codes = """Transaction Id	Transaction Id Time	File
f9a193cdfc7b85d193304b4476068bfd	"00:00:37.94"	user-1
47a9063f57366fb5c44b753de6fd00d4	"00:04:21.91"	user-2
7f7b0014d9a8e2bf8b8ffe896022053d	"00:00:55.68"	user-2-extra
083b0beee70cb2a8a5848747acd19d99	"00:02:20.91"	user-3
435ad5758959790983c52583095645d2	"00:02:18.36"	user-4
7dc2f7c64eeb494f8d583f01f8b227c9	"00:01:16.01"	user-4-extra
954b6639595169b8b3960e84e05d2549	"00:00:58.81"	user-5
c4da6cb61d25ac4b769cc471f0e278f4	"00:05:15.96"	user-6
533c935c169ff4ef093bca2be39beb73	"00:27:41.08"	user-6#2
1dc468a3a003f8b729dc07adc1c7c096	"00:01:17.01"	user-7
356e38369fd586fac5460214ce19f23f	"00:00:39.28"	user-8
8c384c7cb6fc1c82c16e2c24b0dd0c46	"00:28:52.02"	user-8#2
16afe7343c97e91216c39ddac89167e0	"00:01:55.33"	user9
eae9ffaa67c98cf5e954f36ad1cc80c4	"00:18:26.60"	user9#2
31c708ea230a79c3257ff4045fff447d	"00:03:52.23"	user10
8d2e1a475bef2c6563311ee6053ffe1a	"00:23:08.81"	user10#2"""

df_join = pd.read_csv(StringIO(codes), sep='\t')

# Join reference timestamps for think-aloud timestamps
df = df.merge(df_join, how = 'left', on = 'Transaction Id')

df['has_reference_timestamp'] = df['Transaction Id Time'].map(pd.isna)

df['Transaction Id Time Num'] =\
    df['Transaction Id Time'].map(lambda x: np.nan if pd.isna(x) else process_timestamp_zoom(x))

df['Transaction Id Time Num'] = df['Transaction Id Time Num'].ffill() # Sorted by anon id, time!

# Tutortime to reset at recording reset
df['TimeDiff'] = np.where(df['has_reference_timestamp'], (df.Time - df.Time.shift()).map(lambda s: s.total_seconds()), np.nan)
v = df['TimeDiff']
cumsum = v.cumsum().fillna(method='pad')
reset = -cumsum[v.isnull()].diff().fillna(cumsum)
result = v.where(v.notnull(), reset).cumsum()

# Time difference to reference timestamp is used to synchronize with think-aloud transcripts
df['TimeDiffCumsum'] = result 

# Preprocess/fill remaining relative recording times
df['RecordingTime'] = df['TimeDiffCumsum'] + df['Transaction Id Time Num']

# Fill first time point
df['RecordingTime'] =\
    df[['RecordingTime', 'Transaction Id Time Num']]\
        .agg(lambda row: row['Transaction Id Time Num'] if pd.isna(row['RecordingTime']) else row['RecordingTime'], axis = 1)

df[['Transaction Id', 'Participant', 'File', 'Transaction Id Time', 'has_reference_timestamp', 
    'TimeDiff', 'TimeDiffCumsum', 'RecordingTime']].to_csv('inspect.csv', index=False)

# Remove tutor-performed actions
df = df[df['Student Response Subtype'] != 'tutor-performed'].copy()
df = df[df['Input'] != '-1'].copy()

# Preprocessed object for later reference
df_tutor = df.copy().reset_index()

## Step 2: Read in transcriptions

In [ ]:
# In-person sessions
files = [
   'transcripts/user_1_thinkaloud_s23_condition_1.vtt',
    'transcripts/user_2_thinkaloud_s23_condition_2.vtt',
    'transcripts/user_2_thinkaloud_s23_condition_2_extra.vtt',
    'transcripts/user_3_thinkaloud_s23_condition_3.vtt',
    'transcripts/user_4_thinkaloud_s23_condition_4.vtt',
    'transcripts/user_4_thinkaloud_s23_condition_4_extra.vtt',
    'transcripts/user_5_thinkaloud_s23_condition_5.vtt',
]

In [ ]:
# Zoom sessions
zoomfiles = [f'zoomtranscripts/user{i}zoom.vtt' for i in range(6, 10+1)]

In [ ]:
# Conductor names whose utterances in Zoom should be ignored (redacted)
NAMES_TO_SKIP = [
    'Conrad Borchers'
]

def process_timestamp(s):
    x = time.strptime(s, '%M:%S.%f')
    ans = 3600* x.tm_hour + 60* x.tm_min + x.tm_sec + float(s[-4:])
    return ans

def process_zoomfile(f):
    with open(f, 'r') as file:
        lines = [line.rstrip() for line in file]

    # Remove 2 lines preamble
    lines = lines[2:]

    # Sequences are always Number, Time, Name: Speech, Blank
    times, speakers, contents = [], [], []
    for i, line in enumerate(lines):
        if i%4 in [0, 3]:
            continue
        if i%4 == 1:
            times.append(line)
        else:
            elements = line.split(':', 1)
            if len(elements) == 1:
                speakers.append('UNKNOWN')
                contents.append(elements[0])
            else:
                speakers.append(elements[0])
                contents.append(elements[1])

    df = pd.DataFrame({
        'file': [f for _ in range(len(times))],
        'time': times,
        'speaker': speakers,#['Speaker ' + str(hash(s)) for s in speakers], # if user is not de-identified yet
        'content': contents
    })

    # Remove conductor and unknown speakers from analysis
    df = df[~df['speaker'].isin(NAMES_TO_SKIP + ['UNKNOWN'])].copy()
    
    # Standardize format
    df = df.rename(columns={'speaker': 'user'})
    df['user'] = df.user.map(lambda s: s.replace('cmu-', '').replace('-', '_'))
    df = df[['user', 'time', 'content']]

    df['start'] = df.time.map(lambda s: s.split(' --> ')[0])
    df['end'] = df.time.map(lambda s: s.split(' --> ')[1])
   
    df['start'] = df['start'].map(process_timestamp_zoom)
    df['end'] = df['end'].map(process_timestamp_zoom)
    
    df['center'] = df['start'] + (df['end'] - df['start'])/2
    
    return df

In [ ]:
def process_file(f):
    with open(f, 'r') as file:
        lines = [line.rstrip() for line in file]

    # Remove 2 lines preamble
    lines = lines[2:]

    # Sequences are always Number, Time, Name: Speech, Blank
    times, speakers, contents = [], [], []
    for i, line in enumerate(lines):
        if i%3 == 2:
            continue
        if i%3 == 0:
            times.append(line)
        else:
            contents.append(line)
    
    suff = '_extra' if '_extra' in f else ''  

    df = pd.DataFrame({
        'user': [f.split('/')[-1].split('_thinkaloud')[0]+suff for _ in range(len(times))],
        'time': times,
        'content': contents
    })
    
    df['start'] = df.time.map(lambda s: s.split(' --> ')[0])
    df['end'] = df.time.map(lambda s: s.split(' --> ')[1])
   
    df['start'] = df['start'].map(process_timestamp)
    df['end'] = df['end'].map(process_timestamp)
    
    df['center'] = df['start'] + (df['end'] - df['start'])/2
    return df

In [ ]:
# Export
df_inperson = pd.concat([process_file(file) for file in files])
df_zoom = pd.concat([process_zoomfile(file) for file in zoomfiles])
df = pd.concat([df_inperson, df_zoom])
df = df.sort_values(by = ['user', 'start'])

In [ ]:
df_transcript = df.copy().reset_index()
df_transcript_export = df_transcript.copy()

## Step 3: Postprocess and join tutor log data context to transcriptions

In [ ]:
# 2 Add real time based on log synchronization
to_join = pd.read_csv('logs/ds5371_tx_All_Data_7671_2023_0520_042939.txt', sep='\t')
time_ref = df_join.merge(to_join[['Transaction Id', 'Time']], 
                         how='left', on='Transaction Id')[['File', 'Transaction Id Time', 'Time']]
time_ref['File'] = time_ref['File'].map(lambda s: s.replace('-', ''))

In [ ]:
df_transcript_export2 = df_transcript_export.copy()

In [ ]:
from datetime import timedelta
def add_seconds_to_timestamp(timestamp, seconds):
    # Convert the timestamp to a datetime object
    dt = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
    
    # Create a timedelta object with the specified number of seconds
    delta = timedelta(seconds=seconds)
    
    # Add or subtract the timedelta from the datetime object
    new_dt = dt + delta
    
    # Convert the new datetime object back to a string
    new_timestamp = new_dt.strftime("%Y-%m-%d %H:%M:%S")
    
    return new_timestamp

In [ ]:
# The following code itterates through transcriptions to synchronize 
# the relative time of think-aloud transcripts with the absolute
# time of tutor log data, by looking up values in time_ref.
# Conditional logic deals with multiple recording files per user.

df_transcript_export2['start_real'] = ''
df_transcript_export2['end_real'] = ''
for index, row in df_transcript_export2.iterrows():
    has_cut_jump = False
    user = row['user']
    if not 'extra' in user:
        user_ref = user.replace('_', '')
        ref_slice = time_ref[time_ref['File'] == user_ref].copy()
        ref_time = ref_slice['Transaction Id Time'].values[0]
        ref_real = ref_slice['Time'].values[0]
        if user_ref+'#2' in time_ref['File'].values:
            ref_slice = time_ref[time_ref['File'] == user_ref+'#2'].copy()
            ref_time_second = ref_slice['Transaction Id Time'].values[0]
            ref_real_second = ref_slice['Time'].values[0]
            has_cut_jump = True
    else:
        user_ref = user.replace('_', '')
        ref_slice = time_ref[time_ref['File'] == user_ref].copy()
        ref_time = ref_slice['Transaction Id Time'].values[0]
        ref_real = ref_slice['Time'].values[0]
    st = row['start']
    en = row['end']
    if not has_cut_jump:
        st_diff = st - process_timestamp_zoom(ref_time)
        en_diff = en - process_timestamp_zoom(ref_time)
        df_transcript_export2.loc[index, 'start_real'] =\
            add_seconds_to_timestamp(ref_real, st_diff)
        df_transcript_export2.loc[index, 'end_real'] =\
            add_seconds_to_timestamp(ref_real, en_diff)
    else:
        if st > process_timestamp_zoom(ref_time_second):
            st_diff = st - process_timestamp_zoom(ref_time_second)
            df_transcript_export2.loc[index, 'start_real'] =\
                add_seconds_to_timestamp(ref_real_second, st_diff)
        else:
            st_diff = st - process_timestamp_zoom(ref_time)
            df_transcript_export2.loc[index, 'start_real'] =\
                add_seconds_to_timestamp(ref_real, st_diff)
        if en > process_timestamp_zoom(ref_time_second):
            en_diff = en - process_timestamp_zoom(ref_time_second)
            df_transcript_export2.loc[index, 'end_real'] =\
                add_seconds_to_timestamp(ref_real_second, en_diff)
        else:
            en_diff = en - process_timestamp_zoom(ref_time)
            df_transcript_export2.loc[index, 'end_real'] =\
                add_seconds_to_timestamp(ref_real, en_diff)

In [ ]:
df_transcript_export2

In [ ]:
# The following code adds tutor context to transcripts based
# on the now synchonized timestamps and the closest available
# tutor transactions for each utterance.

df_transcript_export2['selection_before'] = ''
df_transcript_export2['action_before'] = ''
df_transcript_export2['input_before'] = ''
df_transcript_export2['feedback_before'] = ''
for index, row in df_transcript_export2.iterrows():
    st = datetime.strptime(row['start_real'], '%Y-%m-%d %H:%M:%S')
    st_delta = st - df_tutor['Time']
    st_delta_pos = st_delta[st_delta >= np.timedelta64(0)]
    if not len(st_delta_pos) == 0:
        i_before = np.argmin(st_delta_pos)
    else:
        i_before = np.argmax(st_delta)
    selection_before = df_tutor['Selection'].values[i_before]
    action_before = df_tutor['Action'].values[i_before]
    input_before = df_tutor.Input.values[i_before]
    feedback_before = df_tutor['Feedback Text'].values[i_before]
    transaction_id_before = df_tutor['Transaction Id'].values[i_before]
    df_transcript_export2.loc[index, 'selection_before'] = selection_before
    df_transcript_export2.loc[index, 'action_before'] = action_before
    df_transcript_export2.loc[index, 'input_before'] = input_before
    df_transcript_export2.loc[index, 'feedback_before'] = feedback_before
    df_transcript_export2.loc[index, 'transaction_id_before'] = transaction_id_before


In [ ]:
# Join platform and problem ID
df_tutor['problem_id'] = df_tutor['Problem Name'].map(lambda s: int(re.sub('[a-zA-Z_-]', '', s)))
df_tutor['platform'] = df_tutor['Problem Name'].map(lambda s: 'Stoich' if 'Stoich' in s else 'ORCCA')

In [ ]:
df_transcript_export3 = df_transcript_export2.merge(
    df_tutor[['Transaction Id', 'platform', 'problem_id']],
    how = 'left', left_on = 'trbransaction_id_before', right_on = 'Transaction Id'
)

In [ ]:
# Export for hand-coding of problem ID and platform 
df_transcript_export3\
    .sort_values(by = ['user', 'start'])\
    .to_csv('transcripts-with-tutor-context.csv', index=False)

## Use the following R code to summarize utterances by in-tutor attempts for hand-coding

```
df_utterance = read.csv("transcripts-with-tutor-context.csv")

df_utterance = 
  df_utterance %>% 
  fill(problem_id) %>% 
  mutate(
    rowNumber = row_number(),
    instance = ifelse(selection_before == lag(selection_before),NA,rowNumber))
    
# Forward fill the missing values in df_utterance$instance
df_utterance$instance <- zoo::na.locf(df_utterance$instance, na.rm = FALSE)

df_utterance_combined = 
  df_utterance %>% 
  group_by(user,platform,problem_id,selection_before,input_before,feedback_before,instance) %>% 
  summarise(utterance_combined = paste(unique(content), collapse = '/'), 
             feedback_combined = paste(unique(feedback_before), collapse='##'),
           input_combined = paste(unique(input_before), collapse='##') ) %>% 
  arrange(user,platform,problem_id,instance)

write.csv(df_utterance_combined, "utterance-for-coding.csv")
```